Build an autoencoder for the Kaggle MNIST training dataset.

Import and preprocess data

1.   Import and preprocess data
2.   Set parameters and define autoencoder backpropagation function
3.   Run a process to train autoencoder
4.   Visualise what the autoencoder has learned

Take the output of that autoencoder and use it to classify the MNIST test set observations.  Submit your results to Kaggle.com.

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
import matplotlib.pyplot as plt

Using TensorFlow backend.


## Import & preprocess data

In [2]:
data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
data.shape

(42000, 785)

In [4]:
x_train=data.drop('label',axis=1)
y_train=data['label']

In [5]:
x_train.shape

(42000, 784)

In [6]:
y_train.shape

(42000,)

In [7]:
test_data.shape

(28000, 784)

In [29]:
data = data.astype('float32') / 255.
test_data = test_data.astype('float32') / 255.

In [8]:
from keras.utils import np_utils

In [9]:
#train = x_train.values.reshape(-1, 28, 28, 1)
#test = test_data.values.reshape(-1, 28, 28, 1)
# target = np_utils.to_categorical(y_train, 10)

In [30]:
# dividing the data into traing and validation

import sklearn
from sklearn import model_selection

x_train, x_val, y_train, y_val = model_selection.train_test_split(
    train, target, test_size=0.20, random_state=42)

In [31]:
print("Number of images in X_train: {}".format(x_train.shape))
print("Number of images in y_train: {}".format(y_train.shape))

Number of images in X_train: (33600, 28, 28, 1)
Number of images in y_train: (33600, 10)


In [32]:
print("Number of images in X_train: {}".format(x_val.shape))
print("Number of images in y_train: {}".format(y_val.shape))

Number of images in X_train: (8400, 28, 28, 1)
Number of images in y_train: (8400, 10)


## Define Autoencoder
### Used Convolutional Autoencoder

In [42]:
conv_encoder = keras.models.Sequential(
    [
     keras.layers.Reshape([28,28,1], input_shape = [28, 28, 1]),
     keras.layers.Conv2D(16, kernel_size = 3, padding = "same", activation="selu"),
     keras.layers.MaxPool2D(pool_size=2),
     keras.layers.Conv2D(32, kernel_size=3, padding="same", activation="selu"),
     keras.layers.MaxPool2D(pool_size=2),
     keras.layers.Conv2D(64, kernel_size=3, padding="same", activation="selu"),
     keras.layers.MaxPool2D(pool_size=2) 
    ]
)
conv_decoder = keras.models.Sequential(
    [
     keras.layers.Conv2DTranspose(32,kernel_size=3, strides=2, padding="valid", activation="selu", input_shape=[3,3,64]),
     keras.layers.Conv2DTranspose(16, kernel_size=3, strides=2, padding="same", activation="selu"),
     keras.layers.Conv2DTranspose(1, kernel_size=3, strides=2, padding="same", activation="sigmoid"),
     keras.layers.Reshape([28,28, 1])                                   
])
conv_ae = keras.models.Sequential([conv_encoder, conv_decoder])

In [43]:
conv_ae.compile(loss="binary_crossentropy", optimizer=keras.optimizers.SGD(lr = 1.5))

In [44]:
conv_ae.fit(x_train, y_train, epochs=10, validation_data=[x_val, y_val])

ValueError: A target array with shape (33600, 10) was passed for an output of shape (None, 28, 28, 1) while using as loss `binary_crossentropy`. This loss expects targets to have the same shape as the output.